In [2]:
import pandas as pd
import os
import pandas as pd
from os import listdir
from os.path import isfile, join
import json
import collections
import re
from tqdm import tqdm
import tarfile 
import time
from collections import Counter

# Importation infos patents

In [3]:
pathData = "C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/data/"
pathYear = pathData + f"/2016/"  # Updates with variable year
jsonNamesYear = [f for f in listdir(pathYear) if isfile(join(pathYear, f))] 
# jsonNamesYear
listYear = range(2006, 2017)

In [4]:
listYear = range(2006, 2017)
citations_Year = {ipc: {
    'application_number': [], 'patent_number': [], 'publication_number': [], 'main_ipc': [], 'decision': []
} for ipc in listYear}

for year in tqdm(listYear):
    pathYear = pathData + f"/{year}/"  # Updates with variable year
    jsonNamesYear = [f for f in listdir(pathYear) if isfile(join(pathYear, f))] 

    # Total number of JSON files
    total_files = len(jsonNamesYear)

    # Creates a list of patents for each IPC class with batch-size tqdm
    for j in tqdm(range(0, total_files)):
        patent_path = pathYear + jsonNamesYear[j]
        if os.path.exists(patent_path) and os.path.getsize(patent_path) > 0:
            with open(patent_path) as f:
                try:
                    d = json.load(f)  # Load JSON into d
                except json.JSONDecodeError:
                    continue
        else:
            print("File does not exist or is empty:", patent_path)
    
        # Creating the lists for the other information
        citations_Year[year]['application_number'].append(d['application_number'])
        citations_Year[year]['patent_number'].append(d['patent_number'])
        citations_Year[year]['publication_number'].append(d['publication_number'])
        citations_Year[year]['main_ipc'].append(d['main_ipcr_label'])
        citations_Year[year]['decision'].append(d['decision'])


  9%|▉         | 1/11 [00:23<03:53, 23.32s/it]


KeyboardInterrupt: 

# Granted patent

In [30]:
filename = "C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/citations_data/data/g_us_patent_citation.tsv"

### patent_id

In [31]:
# Load only the first column (assuming column 0) without loading the full data
df = pd.read_csv(filename, sep='\t', usecols=[0])

C:\Users\edgar\AppData\Local\Temp\ipykernel_14440\1776422390.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename, sep='\t', usecols=[0])


In [32]:
patent_ids = set(df['patent_id'].astype(str))

In [33]:
# Initialize an empty dictionary to hold DataFrames
df_year = {}

# Iterate over the dictionary and create a DataFrame for each year
for year, data in citations_Year.items():
    # Create a DataFrame for the current year
    df_year[year] = pd.DataFrame(data)

In [34]:
df_year[2006]

,application_number,patent_number,publication_number,main_ipc,decision
0,10513493,7648709,US20060110435A1-20060525,A61K39095,ACCEPTED
1,10520174,8138892,US20090219137A1-20090903,H04Q522,ACCEPTED
2,10526092,8139623,US20060099980A1-20060511,H04B700,ACCEPTED
3,10529231,7640195,US20060178956A1-20060810,G07F1900,ACCEPTED
4,10532323,7558262,US20060268873A1-20061130,H04L1256,ACCEPTED
...,...,...,...,...,...
995,11647147,7564254,US20080088333A1-20080417,G01R3126,ACCEPTED
996,11647274,8966270,US20080159299A1-20080703,H04L1256,ACCEPTED
997,11647383,None,US20080064277A1-20080313,B32B502,REJECTED
998,11648421,None,US20080157335A1-20080703,H01L2352,REJECTED


In [35]:
# Check if each 'patent_number' in df1 is in the set of patent_ids
for year in listYear:
    df_year[year]['is_in_patent_id'] = df_year[year]['patent_number'].apply(lambda x: x in patent_ids)


In [36]:
df_year[2006]

,application_number,patent_number,publication_number,main_ipc,decision,is_in_patent_id
0,10513493,7648709,US20060110435A1-20060525,A61K39095,ACCEPTED,False
1,10520174,8138892,US20090219137A1-20090903,H04Q522,ACCEPTED,True
2,10526092,8139623,US20060099980A1-20060511,H04B700,ACCEPTED,True
3,10529231,7640195,US20060178956A1-20060810,G07F1900,ACCEPTED,True
4,10532323,7558262,US20060268873A1-20061130,H04L1256,ACCEPTED,True
...,...,...,...,...,...,...
995,11647147,7564254,US20080088333A1-20080417,G01R3126,ACCEPTED,True
996,11647274,8966270,US20080159299A1-20080703,H04L1256,ACCEPTED,True
997,11647383,None,US20080064277A1-20080313,B32B502,REJECTED,False
998,11648421,None,US20080157335A1-20080703,H01L2352,REJECTED,False


In [37]:
# Initialize a dictionary to hold the results
results = {}

# Process each year
for year in listYear:
    total_count = len(df_year[year])  # Total number of records for the year
    
    # Count occurrences of `is_in_patent_id` and `decision`
    is_in_patent_id_counter = Counter(df_year[year].is_in_patent_id)
    decision_counter = Counter(df_year[year].decision)
    is_in_patent_id_counter_ACCEPTED = Counter(df_year[year].is_in_patent_id[df_year[year].decision=="ACCEPTED"])

    # Calculate percentages for is_in_patent_id
    accepted_count = decision_counter.get("ACCEPTED", 0)
    not_accepted_count = total_count - accepted_count

    results[year] = {
        'in__CitationSource': is_in_patent_id_counter.get(True, 0),
        'not_in_CitationSource': is_in_patent_id_counter.get(False, 0),
        'in__CitationSource_percent': is_in_patent_id_counter.get(True, 0) / total_count * 100,

        'in__CitationSource_ACCEPTED': is_in_patent_id_counter_ACCEPTED.get(True, 0),
        'not_in__CitationSource_ACCEPTED': is_in_patent_id_counter_ACCEPTED.get(False, 0),
        'not_in__CitationSource_percent_ACCEPTED': is_in_patent_id_counter_ACCEPTED.get(True, 0) / (is_in_patent_id_counter_ACCEPTED.get(True, 0)+is_in_patent_id_counter_ACCEPTED.get(False, 0)) * 100,

        # 'is_in_patent_id_False_percent': is_in_patent_id_counter.get(False, 0) / total_count * 100,

        'accepted_count': accepted_count,
        'rejected_count': decision_counter.get("REJECTED", 0),
        # 'rejected_percent': decision_counter.get("REJECTED", 0) / total_count * 100,
        'pending_count': decision_counter.get("PENDING", 0),
        # 'pending_percent': decision_counter.get("PENDING", 0) / total_count * 100,
        'accepted_percent': accepted_count / total_count * 100,
    }

# Convert the results dictionary into a DataFrame
results_df = pd.DataFrame.from_dict(results, orient='index')

# Optional: Reset index for better formatting
results_df.reset_index(inplace=True)
results_df.rename(columns={'index': 'Year'}, inplace=True)

# Display the results DataFrame
results_df

,Year,in__CitationSource,not_in_CitationSource,in__CitationSource_percent,in__CitationSource_ACCEPTED,not_in__CitationSource_ACCEPTED,not_in__CitationSource_percent_ACCEPTED,accepted_count,rejected_count,pending_count,accepted_percent
0,2006,590,410,59.0,590,26,95.779221,616,381,3,61.6
1,2007,583,417,58.3,583,25,95.888158,608,387,5,60.8
2,2008,598,402,59.8,598,48,92.569659,646,348,6,64.6
3,2009,619,381,61.9,619,49,92.664671,668,327,5,66.8
4,2010,622,378,62.2,622,41,93.815988,663,322,15,66.3
5,2011,650,350,65.0,650,46,93.390805,696,284,20,69.6
6,2012,588,412,58.8,588,75,88.687783,663,302,35,66.3
7,2013,602,398,60.2,602,81,88.140556,683,271,46,68.3
8,2014,530,470,53.0,530,67,88.777219,597,261,142,59.7
9,2015,472,528,47.2,472,85,84.739677,557,162,281,55.7


### Citation patent id

In [12]:
df = pd.read_csv(filename, sep='\t', usecols=[2])

In [13]:
citation_patent_ids = set(df['citation_patent_id'].astype(str))

In [14]:
citation_patent_ids

{'7371623',
 '9764161',
 '10392566',
 '8719503',
 '3284678',
 '2808284',
 '4691651',
 '2003/0082022',
 '5348231',
 '8013229',
 '3428680',
 '5700716',
 '1142072',
 '1826454',
 '6295186',
 '7952233',
 '11473246',
 '3582398',
 'D696178',
 '2856550',
 '10859238',
 'D325165',
 'D861726',
 '6011681',
 '8185533',
 '7613758',
 '9449627',
 '4875213',
 'RE36122',
 '2571245',
 'D1018578',
 '4181113',
 '7909507',
 '5295614',
 '2921202',
 '2415933',
 '1899591',
 '10374632',
 '3419222',
 '9499649',
 '1889813',
 'D214456',
 '2136234',
 '8911424',
 '2563845',
 '8250363',
 '5242339',
 '1070387',
 '3958043',
 '10829894',
 '1677393',
 '3402861',
 '3154337',
 '7200933',
 '4121196',
 'RE32342',
 'D244286',
 'D778388',
 '4594982',
 '7934557',
 '8732660',
 '6734851',
 '9049521',
 '998952',
 '7596633',
 'D480176',
 'D445141',
 'D537232',
 '9090486',
 '4948437',
 '2582541',
 '11281969',
 '8241943',
 '5651522',
 '7860115',
 '7215381',
 '10713819',
 'D894899',
 '6646504',
 '8601822',
 '5054971',
 '3692651',
 '31

In [15]:
len(citation_patent_ids)

8789386

In [16]:
df_year[2006]

,application_number,patent_number,publication_number,main_ipc,decision,is_in_patent_id
0,10513493,7648709,US20060110435A1-20060525,A61K39095,ACCEPTED,False
1,10520174,8138892,US20090219137A1-20090903,H04Q522,ACCEPTED,True
2,10526092,8139623,US20060099980A1-20060511,H04B700,ACCEPTED,True
3,10529231,7640195,US20060178956A1-20060810,G07F1900,ACCEPTED,True
4,10532323,7558262,US20060268873A1-20061130,H04L1256,ACCEPTED,True
...,...,...,...,...,...,...
995,11647147,7564254,US20080088333A1-20080417,G01R3126,ACCEPTED,True
996,11647274,8966270,US20080159299A1-20080703,H04L1256,ACCEPTED,True
997,11647383,None,US20080064277A1-20080313,B32B502,REJECTED,False
998,11648421,None,US20080157335A1-20080703,H01L2352,REJECTED,False


In [17]:
# Check if each 'patent_number' in df1 is in the set of patent_ids
for year in listYear:
    df_year[year]['patentNb_in_citaPatent_id'] = df_year[year]['patent_number'].apply(lambda x: x in citation_patent_ids)

In [18]:
# Check if each 'patent_number' in df1 is in the set of patent_ids
for year in listYear:
    df_year[year]['appNb_in_citaPatent_id'] = df_year[year]['application_number'].apply(lambda x: x in citation_patent_ids)

In [19]:
# Check if each 'patent_number' in df1 is in the set of patent_ids
for year in listYear:
    df_year[year]['pubNb_in_citaPatent_id'] = df_year[year]['publication_number'].apply(lambda x: x in citation_patent_ids)

In [42]:
df_year[2006]

,application_number,patent_number,publication_number,main_ipc,decision,is_in_patent_id,patentNb_in_citaPatent_id,appNb_in_citaPatent_id,pubNb_in_citaPatent_id
0,10513493,7648709,US20060110435A1-20060525,A61K39095,ACCEPTED,False,False,False,False
1,10520174,8138892,US20090219137A1-20090903,H04Q522,ACCEPTED,True,True,True,False
2,10526092,8139623,US20060099980A1-20060511,H04B700,ACCEPTED,True,True,True,False
3,10529231,7640195,US20060178956A1-20060810,G07F1900,ACCEPTED,True,True,True,False
4,10532323,7558262,US20060268873A1-20061130,H04L1256,ACCEPTED,True,True,False,False
...,...,...,...,...,...,...,...,...,...
995,11647147,7564254,US20080088333A1-20080417,G01R3126,ACCEPTED,True,True,False,False
996,11647274,8966270,US20080159299A1-20080703,H04L1256,ACCEPTED,True,True,False,False
997,11647383,None,US20080064277A1-20080313,B32B502,REJECTED,False,False,False,False
998,11648421,None,US20080157335A1-20080703,H01L2352,REJECTED,False,False,False,False


# Applied for patents

In [20]:
filename = "C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/citations_data/data/g_us_application_citation.tsv"

### patent_id

In [21]:
# Load only the first column (assuming column 0) without loading the full data
df = pd.read_csv(filename, sep='\t', usecols=[0])

In [22]:
patent_ids = set(df['patent_id'].astype(str))

In [25]:
# Initialize an empty dictionary to hold DataFrames
df_year = {}

# Iterate over the dictionary and create a DataFrame for each year
for year, data in citations_Year.items():
    # Create a DataFrame for the current year
    df_year[year] = pd.DataFrame(data)

In [ ]:
# Check if each 'patent_number' in df1 is in the set of patent_ids
for year in listYear:
    df_year[year]['is_in_patent_id'] = df_year[year]['patent_number'].apply(lambda x: x in patent_ids)

In [29]:
# Initialize a dictionary to hold the results
results = {}

# Process each year
for year in listYear:
    total_count = len(df_year[year])  # Total number of records for the year
    
    # Count occurrences of `is_in_patent_id` and `decision`
    is_in_patent_id_counter = Counter(df_year[year].is_in_patent_id)
    decision_counter = Counter(df_year[year].decision)
    is_in_patent_id_counter_ACCEPTED = Counter(df_year[year].is_in_patent_id[df_year[year].decision=="ACCEPTED"])

    # Calculate percentages for is_in_patent_id
    accepted_count = decision_counter.get("ACCEPTED", 0)
    not_accepted_count = total_count - accepted_count

    results[year] = {
        'in_CitationSource': is_in_patent_id_counter.get(True, 0),
        'not_in_CitationSource': is_in_patent_id_counter.get(False, 0),
        'in_CitationSource_percent': is_in_patent_id_counter.get(True, 0) / total_count * 100,

        'in_CitationSource_ACCEPTED': is_in_patent_id_counter_ACCEPTED.get(True, 0),
        'not_in_CitationSource_ACCEPTED': is_in_patent_id_counter_ACCEPTED.get(False, 0),
        'in_CitationSource_percent_ACCEPTED': is_in_patent_id_counter_ACCEPTED.get(True, 0) / (is_in_patent_id_counter_ACCEPTED.get(True, 0)+is_in_patent_id_counter_ACCEPTED.get(False, 0)) * 100,

        # 'is_in_patent_id_False_percent': is_in_patent_id_counter.get(False, 0) / total_count * 100,

        'accepted_count': accepted_count,
        'rejected_count': decision_counter.get("REJECTED", 0),
        # 'rejected_percent': decision_counter.get("REJECTED", 0) / total_count * 100,
        'pending_count': decision_counter.get("PENDING", 0),
        # 'pending_percent': decision_counter.get("PENDING", 0) / total_count * 100,
        'accepted_percent': accepted_count / total_count * 100,
    }

# Convert the results dictionary into a DataFrame
results_df = pd.DataFrame.from_dict(results, orient='index')

# Optional: Reset index for better formatting
results_df.reset_index(inplace=True)
results_df.rename(columns={'index': 'Year'}, inplace=True)

# Display the results DataFrame
results_df

,Year,in_CitationSource,not_in_CitationSource,in_CitationSource_percent,in_CitationSource_ACCEPTED,not_in_CitationSource_ACCEPTED,in_CitationSource_percent_ACCEPTED,accepted_count,rejected_count,pending_count,accepted_percent
0,2006,473,527,47.3,473,143,76.785714,616,381,3,61.6
1,2007,484,516,48.4,484,124,79.605263,608,387,5,60.8
2,2008,510,490,51.0,510,136,78.947368,646,348,6,64.6
3,2009,570,430,57.0,570,98,85.329341,668,327,5,66.8
4,2010,573,427,57.3,573,90,86.425339,663,322,15,66.3
5,2011,621,379,62.1,621,75,89.224138,696,284,20,69.6
6,2012,587,413,58.7,587,76,88.536953,663,302,35,66.3
7,2013,634,366,63.4,634,49,92.825769,683,271,46,68.3
8,2014,547,453,54.7,547,50,91.624791,597,261,142,59.7
9,2015,511,489,51.1,511,46,91.741472,557,162,281,55.7
